# Implementation of solving algorithm

## Section 2: 2-D Case

TODO

In [1]:
include("group4-2D.jl")
using LinearAlgebra
using SparseArrays 
using StructArrays
using StaticArrays
using SparseArrays 
using StructArrays
using StaticArrays
using StaticRanges
using FastGaussQuadrature
using IterativeSolvers
using Preconditioners
using LinearSolve
using BenchmarkTools
using Profile
using ProfileView
using IncompleteLU
using Plots 
using FastGaussQuadrature
using IterativeSolvers
using Preconditioners
using LinearSolve
using Benchm
using Profile
using ProfileView
using IncompleteLU
using Plots 
using AlgebraicMultigrid: ruge_stuben,smoothed_aggregation, asprec

ArgumentError: ArgumentError: Package AlgebraicMultigrid not found in current path.
- Run `import Pkg; Pkg.add("AlgebraicMultigrid")` to install the AlgebraicMultigrid package.

In [2]:
# function mesh_maker(n)
#     if dim == 1
#         mesh = meshFromGmsh("square-1.msh");
#     elseif dim == 2
#         mesh = meshFromGmsh("square-10.msh");
#     elseif dim == 3
#         mesh = meshFromGmsh("square-100.msh");
#     else
#         throw(DomainError(dim, "this value is not 1,10,100"))
#     end

#     return mesh
# end

In [3]:
# function assembly(mesh)

#     returning_struct = []

#     A = genStiffMat(mesh);
#     f = genVector(mesh, mySourceFct);
#     A, f = handleBoundary!(mesh, A, f);

#     push!(returning_struct, A);
#     push!(returning_struct,f);
#     return returning_struct;
# end

### 1.1 Algorithm comparation

In this section we compare the time spent by different solving algorithm. For the one dimension problem we have chosen to analyze the classical "\\" operator, with and without using the LU decomposition, and two different iterative solvers: $pcg$ : preconditioned conjugate gradient method using the $ILU$ preconditioner and the $MINRES$ method. <br> The conjugate gradient method starts with a guess for the u vector and stops whetever the $residual: r=\|\mathbf{Au-b}\|$ is as near as we initially wanted to be to $0$. <br> The $MINRES  method$  uses the same idea but it doesn't assume that the matrix is positive definite. 

In [4]:
function solve_and_write_time(n)
    
    ret = zeros(8);
    
    #Measure the time of assembly
    LP=buildMatandVec(1/n)
    ret[1]= LP[3]

    A = LP[1]
    f = LP[2]

    #Measure the time to solve the linear system with the cg method
    elapsed_time_solving = @elapsed begin
        preconditioner = ilu(A)
        x = cg(A, f, Pl=preconditioner,abstol=1e-7)
    end
    ret[2] = elapsed_time_solving

    #Measure the time to solve the linear system with the AMG algorithm - smoothed_aggregation
    elpased_time_AMGsa= @elapsed begin
        ml_sa = smoothed_aggregation(A)
        p_sa = aspreconditioner(ml_sa)
    
        x = cg(A, f, Pl = p_sa,abstol=1e-7)
    end
    ret[5]= elpased_time_AMGsa 

    #Measure the time to solve the linear system with the AMG algorithm - ruge_stuben
    elpased_time_AMGrs= @elapsed begin
        ml_rs = ruge_stuben(A)
        p_rs = aspreconditioner(ml_rs)
        x = cg(A, f, Pl = p_rs,abstol=1e-7)
    
    end
    ret[6]= elpased_time_AMGrs
    

    #Measure the time to solve the linear system with the LU decomposition and \ operator
    elapsed_time_LU = @elapsed begin
        luMat = lu(m)
        x = luMat\f
    end
    ret[8]=elapsed_time_LU

    ret[4] = size(f)[1]

    return ret
end


solve_and_write_time (generic function with 1 method)

Now we need to call the function to see how the time changes with the increasing of the dimension of the problem for each algorithm, so we make a loop in which we iterate the dimension and in each iteration we calculate the time to solve the linear problem

In [5]:
# Set the dimensions for the loop (e.g., from 2 to 100000 with a step of 1000)
dimensions = 2:1000:10000
times_assembly = []
times_ilu = []
times_AMGsa = []
times_AMGrs = []
times_LU = []
steps = []

# Solve linear systems and collect elapsed times
for n in dimensions
    time = solve_and_write_time(n)

    push!(times_assembly, time[1])
    push!(times_ilu, time[2])
    push!(times_AMGsa, time[5])
    push!(times_AMGrs, time[6])
    push!(times_LU,time[8])

    push!(steps,time[4])

end

# Plot the graph
plot(steps, times_assembly, xlabel="Dimension", ylabel="Elapsed Time (seconds)", label="Assembly", legend=:topleft)
plot!(steps,times_ilu, label="Solving_ILU")
plot!(steps,times_AMGsa, label="Solving_AMGsa")
plot!(steps,times_AMGrs, label="Solving_AMGrs")
plot!(steps,times_LU, label="Solving_LU")


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.488692s, CPU 0.25s)
Info    : 3013 nodes 6028 elements
Info    : Writing 'square.msh'...
Info    : Done writing 'square.msh'
Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3013 nodes
Info    : 6028 elements
Info    : Done reading 'square.msh'
  0.003465 seconds (17.50 k allocations: 3.718 MiB)


  0.001739 seconds (11.67 k allocations: 1.671 MiB)


  0.001456 seconds (23.30 k allocations: 1.800 MiB)
Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3013 nodes
Info    : 6028 elements
Info    : Done reading 'square.msh'


UndefVarError: UndefVarError: `ilu` not defined